# 載入所需套件

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import time

#隱藏warning
requests.packages.urllib3.disable_warnings()

# 定義去除unicode字串函數

In [2]:
def process(input_str):
    return ''.join([c if ord(c)<=55291 else '_' for c in input_str])

# 定義司法院裁判書關鍵字查詢函數

In [3]:
def law_crawler(name):
    url='https://law.judicial.gov.tw/FJUD/default.aspx'

    #先get()取出會變動的參數
    session=requests.session()
    response=session.get(url,verify=False)
    soup=BeautifulSoup(response.text,'html.parser')

    VIEWSTATE=soup.select_one('#__VIEWSTATE')['value']
    VIEWSTATEGENERATOR=soup.select_one('#__VIEWSTATEGENERATOR')['value']
    VIEWSTATEENCRYPTED=soup.select_one('#__VIEWSTATEENCRYPTED')['value']
    EVENTVALIDATION=soup.select_one('#__EVENTVALIDATION')['value']

    form_data={'__VIEWSTATE':VIEWSTATE,
               '__VIEWSTATEGENERATOR':VIEWSTATEGENERATOR,
               '__VIEWSTATEENCRYPTED':VIEWSTATEENCRYPTED,
               '__EVENTVALIDATION':EVENTVALIDATION,
               'txtKW':name,
               'judtype':'JUDBOOK',
               'whosub':'0',
               'ctl00$cp_content$btnSimpleQry':'送出查詢'}

    #再post餵入form_data
    response=session.post(url,data=form_data,verify=False)
    soup=BeautifulSoup(response.text,'html.parser')
    #爬取q_value值
    q_value=soup.select_one('#hidQID')['value']

    #調整網址q_value值進入判決整理頁面
    url='https://law.judicial.gov.tw/FJUD/qryresultlst.aspx?q={}'.format(q_value)
    response=requests.get(url,verify=False)
    soup=BeautifulSoup(response.text,'html.parser')

    #紀錄總判決頁數，處理頁數可能只有一頁的情況
    try:
        pages=int(soup.select('div.pull-right')[0].text.split(' / ')[1].split(' 頁')[0])
    except:
        pages=1
    
    #最多爬500筆資料(25頁)
    if pages>25:
        pages=25
    
    dfs=[]
    #抓取每頁判決表格
    for page in tqdm(range(1,pages+1)):
        link=url+'&sort=DS&page={}&ot=in'.format(page)
        response=requests.get(link,verify=False)
        try:
            df=pd.read_html(response.text)[0]
            Names=pd.Series([name]*len(df[df.index%2!=0]))
            Contents=df[df.index%2!=0]['裁判日期'].apply(lambda x:process(x)).reset_index(drop=True)
            CaseNos=df[df.index%2==0]['裁判字號 （內容大小）'].apply(lambda x:x.split('第 ')[1].split(' 號')[0]).reset_index(drop=True).astype(int)
            Types=df[df.index%2==0]['裁判案由'].reset_index(drop=True)
            Times=df[df.index%2==0]['裁判日期'].reset_index(drop=True)
            Locations=df[df.index%2==0]['裁判字號 （內容大小）'].apply(lambda x:x[2:4] if ('地方法院' in x) else x[7:9]).str.replace('10','臺北').reset_index(drop=True)
            Titles=df[df.index%2==0]['裁判字號 （內容大小）'].reset_index(drop=True)
            #建立小表格
            df=pd.DataFrame({'name':Names,
                             'time':Times,
                             'location':Locations,
                             'caseNo':CaseNos,
                             'type':Types,
                             'content':Contents,
                             'title':Titles})
            dfs.append(df)
            time.sleep(0.5)
        except:
            None
    #合併成大表格
    df=pd.concat(dfs,ignore_index=True)
    #改成字典格式
    data_list=df.to_dict(orient='records')
        
        
    return df,data_list

# 抓取指定司法院裁判書關鍵字查詢結果

In [4]:
df,data_list=law_crawler('王小明')

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:06<00:00,  1.61s/it]


In [5]:
df.head()

,name,time,location,caseNo,type,content,title
0,王小明,108.12.30,彰化,950,恐嚇危害安全,...時不在場之老闆及同事於事後告知他有綽號「王小明」之人_他才知道有綽號「王小明」之人乙節...,臺灣彰化地方法院 108 年度 易 字第 950 號刑事判決（21K）
1,王小明,108.12.06,高雄,401,毒品危害防制條例,...見本院卷第113、11 9頁__均應依前揭規定宣告沒收。_至被告販賣甲基安非他命予陳致...,臺灣高雄地方法院 108 年度 訴 字第 401 號刑事判決（14K）
2,王小明,108.03.14,新北,3847,侵權行為損害賠償,...回覆略以_當時並無意外停電之情形_有臺灣新北地方檢察署函、公務電話記錄在卷可參_詳刑案...,臺灣新北地方法院 106 年度 訴 字第 3847 號民事判決（33K）
3,王小明,108.02.25,新北,24,詐欺,...限公司回覆略以_當時並無意外停電之情形_有臺灣新北地方檢察署函、公務電話記錄在卷可參_...,臺灣新北地方法院 107 年度 易 字第 24 號刑事判決（24K）
4,王小明,107.05.14,雲林,2782,支付命令,...│ 利息計算方式 ││序號││││││├──┼───┼─────┼──────┼───...,臺灣雲林地方法院 107 年度 司促 字第 2782 號民事其他文書（2K）


In [6]:
data_list

[{'name': '王小明',
  'time': '108.12.30',
  'location': '彰化',
  'caseNo': 950,
  'type': '恐嚇危害安全',
  'content': '...時不在場之老闆及同事於事後告知他有綽號「王小明」之人_他才知道有綽號「王小明」之人乙節_證人王燦堯於警偵訊指稱對其恐嚇之人係綽號「王小明」之人並以照片指認被告係恐嚇其之人_是否真確無誤_即非無疑。且依證人王燦堯、阮建智、顧家榮於本院上開所證_可知阮建智與顧家榮到場時...',
  'title': '臺灣彰化地方法院 108 年度 易 字第 950 號刑事判決（21K）'},
 {'name': '王小明',
  'time': '108.12.06',
  'location': '高雄',
  'caseNo': 401,
  'type': '毒品危害防制條例',
  'content': '...見本院卷第113、11 9頁__均應依前揭規定宣告沒收。_至被告販賣甲基安非他命予陳致安之販毒所得4,000 元_被告供稱已經交付王小明等語_見本院卷第118 頁__且檢察官亦未提出證據證明被告確仍保有此部分犯罪所得_爰不予宣告沒收或追徵_附此敘明。據上論斷_應依刑...',
  'title': '臺灣高雄地方法院 108 年度 訴 字第 401 號刑事判決（14K）'},
 {'name': '王小明',
  'time': '108.03.14',
  'location': '新北',
  'caseNo': 3847,
  'type': '侵權行為損害賠償',
  'content': '...回覆略以_當時並無意外停電之情形_有臺灣新北地方檢察署函、公務電話記錄在卷可參_詳刑案偵卷二第13、130、134頁_。參酌訴外人王小明於刑案偵查中證稱_「就算颱風停電3天_沒有冷氣的狀況下也還不會死」等語_詳刑案偵卷二第3頁背面_。參諸楊繕誠於刑案偵查中自承_「我...',
  'title': '臺灣新北地方法院 106 年度 訴 字第 3847 號民事判決（33K）'},
 {'name': '王小明',
  'time': '108.02.25',
  'location': '新北',
  'caseNo': 24,
  'typ